# Simulating the nucleosynthesis of elements in stars

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy import integrate

Construct a series of differential equations to simulate the nucleosynthesis of elements in stars due to the CNO1 cycle.

The differential equations are given by:

\begin{align}
\frac{d C12}{dt} &= H \left(N15 \right) \langle \sigma v \rangle_{N15 (p,\alpha)} - H \left(C12 \right) \langle \sigma v \rangle_{C12 (p,\gamma)} \\
\frac{d C13}{dt} &= H \left(C12 \right) \langle \sigma v \rangle_{C12 (p,\gamma)} - H \left(C13 \right) \langle \sigma v \rangle_{C13 (p,\gamma)} \\
\frac{d N14}{dt} &= H \left(C13 \right) \langle \sigma v \rangle_{C13 (p,\gamma)} - H \left(N14 \right) \langle \sigma v \rangle_{N14 (p,\gamma)} \\
\frac{d N15}{dt} &= H \left(N14 \right) \langle \sigma v \rangle_{N14 (p,\gamma)} - H \left(N15 \right) \langle \sigma v \rangle_{N15 (p,\alpha)} \\
\end{align}

Where:
- $H(X)$ is the relative abundance of element $X$.
- $\langle \sigma v \rangle_{X (p,\gamma/\alpha)}$ is the reaction rate of the reaction $X (p,\gamma/\alpha)$ determined by the following equation:

\begin{align}
\langle \sigma v \rangle_{X (p,\gamma/\alpha)} &= \int_0^{\infty} v \sigma(v) P(v) dv \\
&= \left( \frac{8}{\pi m_{01}}\right)^{1/2} \frac{1}{(kT)^{3/2}} \int_0^{\infty} E \sigma(E) e^{-E/kT} dE
\end{align}

Where:
- $m_{01}$ is the reduced mass of the system.
- $k$ is the Boltzmann constant.
- $T$ is the temperature of the star's core.
- $E$ is the energy
- $\sigma(E)$ is the cross section of the reaction.

Note: We ignore the reactions involving $O16$ and $N13$ because they decay very quickly relative to the other elements.

In [ ]:
def reaction_rate(temp:float, mass:float, crossSection:float, kB = 8.617333262e-5)->float:
    """
    Calculate the reaction rate (sigma v) for the respective reaction.
    
    Parameters
    ----------
    
    temp : float
        The temperature of the system in Kelvin
    mass : float
        The reduced mass of the system
    crossSection : float
        The cross-section of the reaction
    kB : float
        The Boltzmann constant. Default is 8.617333262e-5 in eV/K
    
    Returns
    -------
    
    rate : float
        The reaction rate
    """
    constant = (8/(np.pi*mass))**(1/2) * 1/(kB * temp)**(3/2)
    
    integral = integrate.quad(
        lambda E: E * np.exp(-E/(kB * temp))*crossSection,
        0, np.inf)
    
    return constant * integral[0]

In [ ]:
def calc_red_mass(m1:float, m2:float)->float:
    """
    Calculate the reduced mass of a system
    
    Parameters
    ----------
    
    m1 : float
        The mass of the first particle
    m2 : float
        The mass of the second particle
    
    Returns
    -------
    
    red_mass : float
        The reduced mass of the system
    """
    return m1*m2/(m1+m2)

In [ ]:
def CNO_cycle(elements:dict, masses:dict, temp=15e6, crossSection = 1)->dict:
    """
    Construct the differential equations for systems of the CNO1 cycle.
    
    Parameters
    ----------
    elements : dict
        Dictionary containing the current values of the initialAbundances.
    masses : dict
        Masses of the initialAbundances in the system.
    temp : float
        The temperature of the system in Kelvin. Default is 15 million Kelvin.
    crossSection : float
        The cross-section of the reaction. Default is 1 for the time being.
        
    Returns
    -------
    dX : dict
        The differential equations for the system.
    """
    
    dC12 = ((reaction_rate(temp, masses['N15'], crossSection) * elements['C12']/elements['H1']) - (reaction_rate(temp, masses['C12'], crossSection) * elements['N15']/elements['H1']))
    
    dC13 = ((reaction_rate(temp, masses['C12'], crossSection) * elements['C12']/elements['H1']) - (reaction_rate(temp, masses['C13'], crossSection) * elements['C13']/elements['H1']))
    
    dN14 = ((reaction_rate(temp, masses['C13'], crossSection) * elements['C13']/elements['H1']) - (reaction_rate(temp, masses['N14'], crossSection) * elements['N14']/elements['H1']))
    
    dN15 = ((reaction_rate(temp, masses['N14'], crossSection) * elements['N14']/elements['H1']) - (reaction_rate(temp, masses['N15'], crossSection) * elements['N15']/elements['H1']))
    
    
    dX = {'C12': dC12, 'C13': dC13, 'N14': dN14, 'N15': dN15, 'H1': 0}

    return dX

In [ ]:
def integrate_system(elements:dict, masses:dict, temp=15e6, crossSection = 1)->tuple:
    """
    Integrate the system of differential equations.
    
    Parameters
    ----------
    elements : dict
        Dictionary containing the initial values of the initialAbundances.
    temp : float
        The temperature of the system in Kelvin. Default is 15 million Kelvin for the core of the Sun.
    crossSection : float
        The cross section of the reaction. Default is 1 for the time being.
    masses : dict
        Masses of the initialAbundances in the system.
        
    Returns
    -------
    times : ndarray
        Array containing the time steps.
    X : ndarray
        Array containing the values of the initialAbundances.
    """
    
    t = np.linspace(0, 1e3, 1000)
    dT = t[1] - t[0]
    X = []
    X.append(elements)
    
    for i in range(1, len(t)):
        dX = CNO_cycle(elements, masses, temp, crossSection)
        
        X.append({key: X[i-1][key] + dT*dX[key] for key in dX})
        elements = X[i]
            
    
    return t, X

In [ ]:
elementMasses = {
    'H1': 1.007825,
    'C12': 12.000000,
    'C13': 13.003355,
    'N14': 14.003074,
    'N15': 15.000109,
    'alpha': 4.002603,
    'proton': 1.007276
} # in atomic mass units

initialAbundances = {
    'H1': 10000,
    'C12': 1000,
    'C13': 0,
    'N14': 0,
    'N15': 0
}


In [ ]:
# plot the results
times, X = integrate_system(initialAbundances, elementMasses)

plt.figure(figsize=(10, 6))
plt.plot(times, [x['H1'] for x in X], label='H1')
plt.plot(times, [x['C12'] for x in X], label='C12')
plt.plot(times, [x['C13'] for x in X], label='C13')
plt.plot(times, [x['N14'] for x in X], label='N14')
plt.plot(times, [x['N15'] for x in X], label='N15')
plt.xlabel('Time (s)')
plt.ylabel('Abundance')
plt.legend()
plt.grid()
plt.show()
